# Stream log handler example

This example will show you how to forward logs into a [Redis Stream](https://redis.io/docs/data-types/streams/) using `rlh.RedisStreamHandler`.

## Logger setup

In [1]:
import logging
from rlh import RedisStreamLogHandler

# define the logger
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Default handler

By default, the `RedisStreamLogHandler` will send the logs to a `redis.Redis` instance (running by default on localhost, port 6379) in a stream named **"logs"**.

### Define a default Redis stream handler and adding the handler to our logger

In [2]:
# define the default Redis stream handler
handler = RedisStreamLogHandler()
# add the handler to the logger
logger.addHandler(handler)

### Emit some logs

In [3]:
logger.info("Some log message")
logger.info("Another log message")
logger.error("An error message!")

INFO:root:Some log message
INFO:root:Another log message
ERROR:root:An error message!


### Retrieve the logs emited

In [4]:
from redis import Redis

r = Redis(decode_responses=True)
r.xrange("logs", "-", "+")

[('1675346777654-0',
  {'msg': 'Some log message',
   'levelname': 'INFO',
   'created': '1675346777.652662'}),
 ('1675346777655-0',
  {'msg': 'Another log message',
   'levelname': 'INFO',
   'created': '1675346777.6544297'}),
 ('1675346777655-1',
  {'msg': 'An error message!',
   'levelname': 'ERROR',
   'created': '1675346777.6552315'})]

## Custom stream name

By default, the logs are saved in a stream named "logs", you can however change this by setting the `stream_name` parameter.

In [5]:
# handler with a custom stream name
handler = RedisStreamLogHandler(stream_name="custom")

## Change saved fields

By default the logs emitted are saved as a dict with the following fields:

- msg : the log message
- levelname : the log level
- created : the timestamp when the log has been created

But those fields can be tunned by specifying the `fields` parameter of `RedisStreamLogHandler`. The fields specified must be valid `LogRecord` attributes (you can see the list of valid attributes in [Python logging documentation](https://docs.python.org/3/library/logging.html#logrecord-attributes)).

In [6]:
# remove the previous handler
logger.removeHandler(handler)

# create a handler with fields msg, lineno and name
handler = RedisStreamLogHandler(stream_name="custom_fields", fields=["msg", "lineno", "name"])
logger.addHandler(handler)

In [7]:
logger.info("Some log message")
logger.info("Another log message")
logger.error("An error message!")

INFO:root:Some log message
INFO:root:Another log message
ERROR:root:An error message!


In [8]:
r.xrange("custom_fields", "-", "+")

[('1675346777811-0',
  {'msg': 'Some log message', 'lineno': '1', 'name': 'root'}),
 ('1675346777812-0',
  {'msg': 'Another log message', 'lineno': '2', 'name': 'root'}),
 ('1675346777813-0',
  {'msg': 'An error message!', 'lineno': '3', 'name': 'root'})]

The dict saved for each log now contains the custom fields specified earlier.

## Save logs as pickle format

Rather than saving only some of the LogRecord attributes, you can save the whole object in their [pickle format](https://docs.python.org/3/library/pickle.html). This can be usefull if you need to re-use the logs in another Python program (pickle format is Python specific).

In [9]:
# remove the previous handler
logger.removeHandler(handler)

# create a handler that saves logs as pickle format
handler = RedisStreamLogHandler(stream_name="pkl_logs", as_pkl=True)
logger.addHandler(handler)

In [10]:
logger.info("Some log message")

INFO:root:Some log message


In [11]:
import pickle

r = Redis()
record = pickle.loads(r.xrange("pkl_logs", "-", "+")[0][1][b"pkl"])
record.getMessage()

'Some log message'